In [ ]:
import pandas as pd
import torch
import string
import sys 
from collections import defaultdict
from torch import nn

def p(thing):
    sys.stdout.write(thing)


def isnan(x):
    if type(x) == str:
        return False
    try:
        int(x)
        return False
    except:
        return True


class pbar:
    def __init__(self, length, total, frac=False):
        self.length = length
        self.total = total
        if not frac:
            p('|' + '.' * length + '|')
        else:
            pass
        self.count = 0
        self.thresh = total // length
        self.all_count = 0
        self.n = 0

    def frac(self):
        if self.n == self.total - 1:
            p('\n')
            return
        if self.n > 0:
            p('\b' * self.len)
        string = f'{self.n}/{self.total}'
        p(string)
        self.len = len(string)
        self.n += 1


class fbar:
    def __init__(self, length):
        self.length = length - 1
        self.count = 0
        self.slen = 0

    def step(self):
        if self.count == self.length:
            p('\b' * self.slen)
            print(f'{self.count}/{self.length}')
            return
        p('\b' * self.slen)
        s = f'{self.count}/{self.length} '
        self.slen = len(s)
        p(s)
        self.count += 1


newsdf = pd.read_csv('./corona_fake.csv')
titles = newsdf['title']
text = newsdf['text']
labels = newsdf['label']
letters = string.ascii_letters + ' !0123456789?'


def enum1(x):
    for i in range(len(x)):
        yield (i + 1, x[i])


vocab = {letter: i for i, letter in enum1(letters)}
vocab = defaultdict(lambda: 0, vocab)
#print(vocab[''],vocab['7'])


def word2tensor(word):
    tens = [vocab[i] for i in word]
    if len(tens)>0:
      return torch.tensor(tens)
    else:
      return torch.tensor([0])


def stack(data):
    result = []
    for i in range(len(data)):
        try:
            result.append(word2tensor(data[i]))
        except:
            result.append(torch.zeros(1))
    return result


tensortext = stack(text)
tensorlabel = []
for label in labels:
    if label == 'Fake' or label == 'fake':
        tensorlabel.append(torch.zeros(1))
    elif label == 'TRUE':
        tensorlabel.append(torch.ones(1))
    else:
        # print(label,'n')
        tensorlabel.append(-1)
tensortitles = stack(titles)


def greatest_len(x):
    great = len(x[0])
    for i in x:
        if len(i) > great:
            great = i
        return great


def pad_to(x, length):
  if len(x)>=length:
    return x
  return torch.nn.functional.pad(x, (0, length - len(x)))


def clean(thing, length):
    new = []
    for sentence in thing:
        new.append(pad_to(sentence, length))
    return new


titlemax = 150
textmax = 400
ptext = clean(tensortext, textmax)
ptitle = clean(tensortitles, titlemax)
len(ptitle), len(ptext), len(tensorlabel)
all_data = []


def batcher(x, y, batch):
    c = batch
    re = []
    for i in range(0, len(x), batch):
        re.append((x[i:c], y[i:c]))
    return re


for j in range(len(ptitle)):
    if tensorlabel[j] < 0:
        # print(tensorlabel[j])
        pass
    elif sum(ptext[j]).sum() == 0 and sum(ptitle[j]).sum() == 0:
        pass
    else:
        all_data.append((ptitle[j], ptext[j], tensorlabel[j]))
train_data = all_data[264:]
val_data = all_data[:264]

def list2tensor(l):
  out=torch.empty(len(l),*l[0].shape)
  return torch.cat(l,out=out)
# print('done')
import math
split = 100
val_titles = list(titles)[:split]
val_text = list(text)[:split]
train_titles = list(titles)[split:]
train_labels = tensorlabel[split:]
train_text = list(text)[split:]
vali_labels=list(tensorlabel)[:split]
all_text=[(text[i],tensorlabel[i]) for i in range(len(text)) if not isnan(text[i]) and tensorlabel[i]>-1]
pro_train_text,pro_val_text=torch.utils.data.random_split(all_text,[len(all_text)-264,264])
with open('./stopwords.txt') as st:
  for i in st:
    stopwords=eval(i)
def remove_stops(x,tolist=False):
  if tolist:
    return list(filter(lambda x: x not in stopwords,x))
  else:
    return filter(lambda x: x not in stopwords,x)
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
print(torch.cuda.is_available())
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
class bconvs(nn.Module):
  def __init__(self,embed=30,c1=10,mid=100,out=1,kernal=3):
    super().__init__()
    self.embedding=nn.EmbeddingBag(len(vocab)+1,embed,sparse=True)
    self.conv=nn.Conv1d(1,1,kernal*embed)
    self.gru=nn.GRU(c1,mid)
    self.fc=nn.Linear(mid,out)
    self.pool=nn.AvgPool1d(3)
    self.kernal=kernal
  def forward(self,x):
    words=torch.stack([self.embedding(word2tensor(i).reshape(1,-1).to(device)) for i in x.split(' ')]).reshape(1,1,-1).to(device)
    while words.size(-1)/self.kernal>1:
      words=self.conv(words)
    return torch.sigmoid(torch.max(words).reshape(-1))
def yt(ob,data, vals, tlabel,vald=None,lr=0.01,optim='sgd'):
  if vald is None:
        vald = data
  model = ob().to(device)
  lof = torch.nn.BCELoss()
  if optim=='sgd':
    o = torch.optim.SGD(model.parameters(), lr)
  else:
    o=torch.optim.Adam(model.parameters(),lr)
  for k in range(25):
      b = fbar(len(data))
      for i in range(len(data)):
          b.step()
          if not isnan(data[i]) and int(tlabel[i]) > -1:
              pred = model(data[i])
              target=tlabel[i].to(device)
              loss = lof(pred.reshape(-1), target)
              model.zero_grad()
              o.zero_grad()
              loss.backward()
              o.step()
      # validation
      model.eval()
      count = 0
      bad = 0
      with torch.no_grad():
          for i in range(vals):
              pick = random.randint(0, len(vald) - 1)
              if not isnan(vald[pick]) and int(tensorlabel[pick]) > -1:
                  pre = model(vald[pick])
                  #print(vald[pick], torch.round(pre), int(tensorlabel[pick]))
                  if torch.round(pre) == tensorlabel[pick]:
                      count += 1
              else:
                  bad += 1
      model.train()
      valid_points = vals - bad
      print(f'Validation accuracy: {count / valid_points}')
      print(loss, f' epoch: {k}')
yt(bconvs,train_text,len(val_text),train_labels,val_text,0.006)

In [ ]:
def traintext(ob,train,tlabels,val_text,val_labels,epochs,optim='sgd',lr=0.01):
  model=ob()
  if optim=='sgd':
    o=torch.optim.SGD(model.parameters(),lr)
  else:
    o=torch.optim.Adam(model.parameters(),lr)
  for epoch in range(epochs):
    p=fbar(len(train))
    for i in range(len(train)):
      p.step()
      if not isnan(train[i]) and tlabels[i]>-1:
        pred=model(train[i])
        loss=lossf(pred,tlabels[i])
        model.zero_grad()
        o.zero_grad()
        loss.backward()
        o.step()
    model.eval()
    with torch.no_grad():
      count=0
      bad=0
      for j in range(len(val_text)):
        if not isnan(val_text[j]) and val_labels[j]>-1:
          if torch.round(model(val_text[j]))==val_labels[j]:
            count+=1
        else:
          bad+=1
    model.train()
    print(count/(len(val_text)-bad),f' epoch: {epoch}')
    print(loss)

def train_cuda_gen(ob,train,tlabels,val_text,val_labels,epochs,optim='sgd',lr=0.01,loss='bce'):
  try:
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model=ob().to(device)
    if loss=='bce':
      lossf=nn.BCELoss()
      fil_func=lambda x: x
    elif loss=='logs':
      lossf=nn.BCEWithLogitsLoss()
      fil_func=lambda x:torch.sigmoid(x)
    else:
      lossf=nn.MSELoss()
      fil_func=lambda x:x
    if optim=='sgd':
      o=torch.optim.SGD(model.parameters(),lr)
    elif optim=='adam':
      o=torch.optim.Adam(model.parameters(),lr)
    elif optim=='sadam':
      o=torch.optim.SparseAdam(model.parameters(),lr)
    for epoch in range(epochs):
      p=fbar(len(train))
      for i in range(len(train)):
        p.step()
        if not isnan(train[i]) and tlabels[i]>-1:
          pred=model(train[i])
          target=tlabels[i].float().to(device)
          loss=lossf(pred,target)
          model.zero_grad()
          o.zero_grad()
          loss.backward()
          o.step()
      model.eval()
      with torch.no_grad():
        count=0
        bad=0
        for j in range(len(val_text)):
          if not isnan(val_text[j]) and val_labels[j]>-1:
            pred=model(val_text[j])
            #print(torch.round(torch.sigmoid(pred)),val_labels[j])
            if torch.round(fil_func(pred))==val_labels[j].to(device):
              count+=1
          else:
            bad+=1
      model.train()
      print(count/(len(val_text)-bad),f' epoch: {epoch}')
      print(loss)
    return model
  except KeyboardInterrupt:
    return model
def into2(x):
  out=torch.tensor([[0,0]])
  out[0,int(x)]=1
  return out.t()
def two_cat(ob,train,tlabels,val_text,val_labels,epochs,optim='sgd',lr=0.01):
  device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model=ob().to(device)
  lossf=nn.CrossEntropyLoss()
  if optim=='sgd':
    o=torch.optim.SGD(model.parameters(),lr)
  else:
    o=torch.optim.Adam(model.parameters(),lr)
  for epoch in range(epochs):
    p=fbar(len(train))
    for i in range(len(train)):
      p.step()
      if not isnan(train[i]) and tlabels[i]>-1:
        pred=model(train[i])
        target=tlabels[i].to(device)
        print(into2(target).size(0))
        loss=lossf(pred,into2(target))
        model.zero_grad()
        o.zero_grad()
        loss.backward()
        o.step()
    model.eval()
    with torch.no_grad():
      count=0
      bad=0
      for j in range(len(val_text)):
        if not isnan(val_text[j]) and val_labels[j]>-1:
          pred=torch.sigmoid(model(val_text[j]))
          if torch.round(torch.sigmoid(pred))==val_labels[j].to(device):
            count+=1
        else:
          bad+=1
    model.train()
    print(count/(len(val_text)-bad),f' epoch: {epoch}')
    print(loss)
def train_better(ob,trains,vals,epochs,optim='sgd',lr=0.01,loss='bce'):
  try:
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model=ob().to(device)
    if loss=='bce':
      lossf=nn.BCELoss()
      fil_func=lambda x: x
    elif loss=='logs':
      lossf=nn.BCEWithLogitsLoss()
      fil_func=lambda x:torch.sigmoid(x)
    else:
      lossf=nn.MSELoss()
      fil_func=lambda x:x
    if optim=='sgd':
      o=torch.optim.SGD(model.parameters(),lr)
    elif optim=='adam':
      o=torch.optim.Adam(model.parameters(),lr)
    elif optim=='sadam':
      o=torch.optim.SparseAdam(model.parameters(),lr)
    for epoch in range(epochs):
      p=fbar(len(trains))
      for i,j in trains:
        p.step()
        pred=model(i)
        target=j.float().to(device)
        loss=lossf(pred.float(),target)
        model.zero_grad()
        o.zero_grad()
        loss.backward()
        o.step()
      model.eval()
      with torch.no_grad():
        count=0
        for vi,vt in vals:
          pred=model(vi)
          if torch.round(fil_func(pred))==vt.to(device):
              count+=1
      model.train()
      print(count/len(vals),f' epoch: {epoch}')
      print(loss)
    return model
  except KeyboardInterrupt:
    return model

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#loss goes to 0, accuracy does not change at all
#not good, titles are too sparse
class gru2(nn.Module):
  def __init__(self,embed=20,mid=100,out=1):
    super().__init__()
    self.embed=nn.EmbeddingBag(len(vocab)+1,embed)
    self.gru=nn.GRU(embed,mid,num_layers=2,dropout=0.6)
    self.fc=nn.Linear(2*mid,out)
    self.d=nn.Dropout(p=0.64)
    self.mid=mid
  def forward(self,t):
    w=torch.stack([self.embed(word2tensor(i).unsqueeze(0).to(device)) for i in remove_stops(t.split(' '))]).to(device)
    #
    o,p=self.gru(w,torch.zeros(2,1,self.mid).to(device))
    p=self.d(p)
    return self.fc(p.reshape(1,-1)).reshape(-1)
piss=train_better(gru2,train_titles,train_labels,val_titles,vali_labels,25,lr=0.009,optim='sgd',loss='logs')

0/1063 1/1063 2/1063 3/1063 4/1063 5/1063 6/1063 7/1063 8/1063 9/1063 10/1063 11/1063 12/1063 13/1063 14/1063 15/1063 16/1063 17/1063 18/1063 19/1063 20/1063 21/1063 22/1063 23/1063 24/1063 25/1063 26/1063 27/1063 28/1063 29/1063 30/1063 31/1063 32/1063 33/1063 34/1063 35/1063 36/1063 37/1063 38/1063 39/1063 40/1063 41/1063 42/1063 43/1063 44/1063 45/1063 46/1063 47/1063 48/1063 49/1063 50/1063 51/1063 52/1063 53/1063 54/1063 55/1063 56/1063 57/1063 58/1063 59/1063 60/1063 61/1063 62/1063 63/1063 

In [ ]:
from collections import OrderedDict
new=OrderedDict({i:torch.zeros(*j.shape) for i,j in a.named_parameters()})
for param in a.parameters():
  p=nn.Parameter(torch.zeros(*p.shape))
  print(p)
a.load_state_dict(new,strict=False)
list(a.parameters())


In [ ]:
with open('./stopwords.txt') as st:
  for i in st:
    stopwords=eval(i)
def remove_stops(x,tolist=False):
  if tolist:
    return list(filter(lambda x: x not in stopwords,x))
  else:
    return filter(lambda x: x not in stopwords,x)


In [ ]:
'''jingle bells batman smells robin laid an egg/ Santa got a .44 and shot him in the head/Barbie dolls,  barbie dolls tried to save his life/ teddy bear, teddy bear, stabbed him with a knife'''

'jingle bells batman smells robin laid an egg/ Santa got a .44 and shot him in the head/Barbie dolls,  barbie dolls tried to save his life/ teddy bear, teddy bear, stabbed him with a knife'

In [ ]:
import math
#bad
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
class manylin(nn.Module):
  def __init__(self,embed=30,out=1,mid=100,kernal=14):
    super().__init__()
    self.embed=nn.EmbeddingBag(len(vocab)+1,embed,sparse=True)
    self.fc=nn.Linear(embed*kernal,mid)
    self.rnn=nn.GRU(mid,out)
    self.step=kernal*embed
    self.mid=mid
    self.kernal=kernal
    self.embed_d=embed
  def forward(self,ins):
    w=torch.stack([self.embed(word2tensor(i).unsqueeze(0).to(device)) for i in remove_stops(ins.split(' '))])
    flat=w.reshape(-1)
    count=self.step
    outs=[]
    h=torch.zeros(1,self.mid)
    #overlapping embeddings gets to 60%, kernal of 9
    #bigger kernal means more accuracy but higher and increasing loss
    iters=len(flat)-1
    iterss=iters if iters>0 else 1
    for i in range(0,iterss,self.embed_d):
      input=pad_to(flat[i:count],self.step)
      h=self.fc(input.unsqueeze(0))
      outs.append(h)
      count+=self.embed_d
    o,e=self.rnn(torch.stack(outs))

    return torch.sigmoid(e.reshape(-1))

manylin_model=train_cuda_gen(manylin,train_titles,train_labels,train_titles,train_labels,25,lr=7.9e-6,optim='sgd',loss='bce')

0/1063 1/1063 2/1063 3/1063 4/1063 5/1063 6/1063 7/1063 8/1063 9/1063 10/1063 11/1063 12/1063 13/1063 14/1063 15/1063 16/1063 17/1063 18/1063 19/1063 20/1063 21/1063 22/1063 23/1063 24/1063 25/1063 26/1063 27/1063 28/1063 29/1063 30/1063 31/1063 32/1063 33/1063 34/1063 35/1063 36/1063 37/1063 38/1063 39/1063 40/1063 41/1063 42/1063 43/1063 44/1063 45/1063 46/1063 47/1063 48/1063 49/1063 50/1063 51/1063 52/1063 53/1063 54/1063 55/1063 56/1063 57/1063 58/1063 59/1063 60/1063 61/1063 62/1063 63/1063 

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
class convstepper(nn.Module):
  def __init__(self,step,mid,stride=1,pad=False,flatten=False):
    r'''yeet'''
    super().__init__()
    self.step=step
    self.stride=stride
    self.gru=nn.GRUCell(self.step,mid)
    self.pad=pad
    self.flatten=flatten
  def forward(self,x):
    count=self.step
    features=[]
    for i in range(0,len(x),self.stride):
      input=pad_to(x[i:count],self.step)
      if self.pad:
        input=nn.functional.pad(input,(self.step-1,0))
      features.append(self.gru(input.reshape(1,-1)))
      count+=self.stride
    if self.flatten:
      return torch.stack(features).reshape(-1)
    return torch.stack(features)

class RCNN(nn.Module):
  def __init__(self,embed=20,mid=100,out=1):
    super().__init__()
    self.embedding=nn.EmbeddingBag(len(vocab)+1,embed)
    self.cs=nn.Sequential(
        convstepper(11*embed,5,stride=embed,flatten=True),
        convstepper(15,3,stride=7,flatten=True),
        convstepper(8,2,stride=5,flatten=True),
        convstepper(4,1,3)
        )
    self.rnn=nn.RNN(1,mid)
    self.fc=nn.Linear(mid,out)
  def forward(self,ins):
    w=torch.stack([self.embedding(word2tensor(i).unsqueeze(0).to(device)) for i in remove_stops(ins.split(' '))])
    f=w.reshape(-1)
    p=self.cs(f)
    _,h=self.rnn(p)
    return self.fc(h).reshape(-1)
train_cuda_gen(RCNN,train_text,train_labels,train_titles,train_labels,25,lr=7.9e-6,optim='sgd',loss='logs')

0/1063 1/1063 2/1063 3/1063 4/1063 5/1063 6/1063 7/1063 8/1063 9/1063 10/1063 11/1063 12/1063 13/1063 14/1063 15/1063 16/1063 17/1063 18/1063 19/1063 20/1063 21/1063 22/1063 23/1063 24/1063 25/1063 26/1063 27/1063 28/1063 29/1063 30/1063 31/1063 32/1063 33/1063 34/1063 35/1063 36/1063 37/1063 38/1063 39/1063 40/1063 41/1063 42/1063 43/1063 44/1063 45/1063 46/1063 47/1063 48/1063 49/1063 50/1063 51/1063 52/1063 53/1063 54/1063 55/1063 56/1063 57/1063 58/1063 59/1063 60/1063 61/1063 62/1063 63/1063 

RCNN(
  (embedding): EmbeddingBag(152, 20, mode=mean)
  (cs): Sequential(
    (0): convstepper(
      (gru): GRUCell(220, 5)
    )
    (1): convstepper(
      (gru): GRUCell(15, 3)
    )
    (2): convstepper(
      (gru): GRUCell(8, 2)
    )
    (3): convstepper(
      (gru): GRUCell(4, 1)
    )
  )
  (rnn): RNN(1, 100)
  (fc): Linear(in_features=100, out_features=1, bias=True)
)

In [ ]:
t=0
f=0
al=0
for i,j in zip(train_titles,train_labels):
  if not isnan(i) and j>-1:
    al+=1
  if j==0 and not isnan(i):
    f+=1
title_chance=1-(f/al)
print(title_chance)
tt=0
tf=0
tal=0
for l,k in zip(val_text,vali_labels):
  if not isnan(l) and j>-1:
    tal+=1
  if k==0 and not isnan(l):
    tf+=1

text_chance=1-(tf/tal)
print(text_chance)

0.5275590551181102
0.47959183673469385


In [ ]:
>>> encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8)
>>> src = torch.rand(1,1, 512)
>>> out = encoder_layer(src)
out.shape

torch.Size([1, 1, 512])

In [ ]:

class minitransformer(nn.Module):
  def __init__(self,embed=20,mid=100,out=1):
    super().__init__()
    self.embed=nn.EmbeddingBag(len(vocab)+1,embed)
    self.atten=nn.TransformerEncoderLayer(embed,10)
    self.gru=nn.GRU(embed,mid)
    self.fc=nn.Linear(mid,out)
  def forward(self,x):
    w=torch.stack([self.embed(word2tensor(i).unsqueeze(0)) for i in remove_stops(x)]).to(device)
    attened=self.atten(w)
    o,h=self.gru(attened)
    return self.fc(h).reshape(-1)

hp=train_cuda_gen(minitransformer,train_text,train_labels,train_text,train_labels,25,lr=7.9e-6,optim='sgd',loss='logs')

0/1063 

In [ ]:
import torch
from torch import nn

ad=nn.AdaptiveMaxPool1d(3)
k=torch.randn(1,1,7)
print(k)
ad(k).shape

tensor([[[-1.4093, -0.1417, -0.4115,  0.9663, -1.8704, -0.3620,  0.3114]]])


torch.Size([1, 1, 3])

In [ ]:
class mhatten(nn.Module):#thin
  def __init__(self,in_shape,qkv_dim,heads):
    super().__init__()
    self.qw=nn.ModuleList(nn.Linear(in_shape,qkv) for h in heads)
    self.kw=nn.ModuleList(nn.Linear(in_shape,qkv) for h in heads)
    self.vw=nn.ModuleList(nn.Linear(in_shape,qkv) for h in heads)
    self.heads=heads
    self.fc=nn.Linear(qkv*(in_shape/heads),in_shape)
    self.qkv=qkv
    self.heads=heads
  def forward(self,x):
    outs=[]
    #x shape i,j  out shape i,qkv
    count=self.heads
    x=x.reshape()
    for i in range(len(self.qw)):
      k=self.kw[i](x[:,i*heads:count,:])
      q=self.qw[i](x[:,i*heads:count,:])
      v=self.vw[i](x[:,i*heads:count,:])
      scores=torch.bmm(k,q.t())/(x.size(-1)**0.5)
      normed=torch.softmax(scores,dim=-1)
      outs.append(torch.bmm(normed,x))
      count+=heads
    return self.fc(torch.stack(outs))

def expand(x,depth):
  return torch.stack([x for i in range(depth)])
class Thiccatten(nn.Module):
  def __init__(self,k,heads):
    super().__init__()
    self.qw=nn.Linear(k,k*heads)
    self.kw=nn.Linear(k,k*heads)
    self.vw=nn.Linear(k,k*heads)
    self.fc=nn.Linear(k*heads,k)
    self.heads=heads
  def forward(self,x):
    b,t,k=x.size()
    h=self.heads
    q=self.qw(x).view(b,t,h,k)
    key=self.kw(x).view(b,t,h,k)
    v=self.vw(x).view(b,t,h,k)
    keys = key.transpose(1, 2).contiguous().view(b * h, t, k)
    queries = q.transpose(1, 2).contiguous().view(b * h, t, k)
    values = v.transpose(1, 2).contiguous().view(b * h, t, k)
    keys=keys/(k**0.25)
    queries=queries/(k**0.25)
    dot=torch.bmm(keys,queries.transpose(1,2))
    scaled_dot=torch.softmax(dot,dim=2)
    out = torch.bmm(scaled_dot, values).view(b, h, t, k)
    out=out.transpose(1,2).contiguous().view(b,t,k*h)
    return self.fc(out)

class tblock(nn.Module):
  def __init__(self, k, heads):
    super().__init__()

    self.attention = Thiccatten(k, heads=heads)

    self.norm1 = nn.LayerNorm(k)
    self.norm2 = nn.LayerNorm(k)

    self.ff = nn.Sequential(
      nn.Linear(k, 4 * k),
      nn.ReLU(),
      nn.Linear(4 * k, k))

  def forward(self, x):
    attended = self.attention(x)
    x = self.norm1(attended + x)
    
    fedforward = self.ff(x)
    return self.norm2(fedforward + x)

class c_transformer(nn.Module):
  def __init__(self,heads=8,depth=7,word_embed=20,max_seq=6000):
    super().__init__()
    self.transformers=nn.Sequential(*[tblock(word_embed,heads) for i in range(depth)])
    self.w_embed=nn.EmbeddingBag(len(vocab)+1,word_embed)
    self.pos_embed=nn.Embedding(max_seq+1,word_embed)
    self.fc=nn.Linear(word_embed,1)
  def forward(self,x):
    w=torch.stack([self.w_embed(word2tensor(i).unsqueeze(0)) for i in remove_stops(x.split(' '))]).transpose(0,1).to(device)
    b,t,k=w.size()
    try:
      pos_embeddings=self.pos_embed(torch.arange(t)).expand(b,t,k)
    except:
      print(w.size(),x)
    attended=self.transformers(pos_embeddings+w)
    classes=self.fc(attended).mean(dim=1)
    return torch.sigmoid(classes.reshape(-1))
f=0
for i,j in pro_val_text:
  if j==0:
    f+=1
print(f/len(pro_val_text))
hp2=train_better(c_transformer,pro_train_text,pro_val_text,25,lr=2.7e-5,optim='adam',loss='bce')

0.5151515151515151
0/886 1/886 2/886 3/886 4/886 5/886 6/886 7/886 8/886 9/886 10/886 11/886 12/886 13/886 14/886 15/886 16/886 17/886 18/886 19/886 20/886 21/886 22/886 23/886 24/886 25/886 26/886 27/886 28/886 29/886 30/886 31/886 32/886 33/886 34/886 35/886 36/886 37/886 38/886 39/886 40/886 41/886 42/886 43/886 44/886 45/886 46/886 47/886 48/886 49/886 50/886 51/886 52/886 53/886 54/886 55/886 56/886 57/886 58/886 59/886 60/886 61/886 62/886 63/886 64/886 65/886 66/886 67/886 68/886 69/886 70/886 71/886 

In [ ]:
#print(hp1.state_dict())
tv2=c_transformer()
tv2.load_state_dict(hp1.state_dict())
hp2=train_better(c_transformer,pro_train_text,pro_val_text,25,lr=2.7e-5,optim='adam',loss='bce')

0/886 1/886 2/886 3/886 4/886 5/886 6/886 7/886 8/886 9/886 10/886 11/886 12/886 13/886 14/886 15/886 16/886 17/886 18/886 19/886 20/886 21/886 22/886 23/886 24/886 25/886 26/886 27/886 28/886 29/886 30/886 31/886 32/886 33/886 34/886 35/886 36/886 37/886 38/886 39/886 40/886 41/886 42/886 43/886 44/886 45/886 46/886 47/886 48/886 49/886 50/886 51/886 52/886 53/886 54/886 55/886 56/886 57/886 58/886 59/886 60/886 61/886 62/886 63/886 64/886 65/886 66/886 67/886 68/886 69/886 70/886 71/886 72/886 

In [ ]:
class gru2(nn.Module):
  def __init__(self,embed=20,mid=100,out=1):
    super().__init__()
    self.embed=nn.EmbeddingBag(len(vocab)+1,embed)
    self.gru=nn.GRU(embed,mid,dropout=0.6)
    self.fc=nn.Linear(mid,out)
    self.d=nn.Dropout(p=0.64)
    self.mid=mid
  def forward(self,t):
    w=torch.stack([self.embed(word2tensor(i).unsqueeze(0).to(device)) for i in remove_stops(t.split(' '))]).to(device)
    o,p=self.gru(w)
    p=self.d(p)
    return torch.sigmoid(self.fc(p.reshape(1,-1)).reshape(-1))

In [ ]:
torch.save(hp2,'model_75v2')
files.download('model_75v2')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
f=0
for i,j in pro_val_text:
  if j==0:
    f+=1
print(f/len(pro_val_text))
from google.colab import files
torch.save(hp1.state_dict(),'model276')
files.download('model276')

0.5151515151515151


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
class gc(nn.Module):
  def __init__(self,embed=20,out=1,mid=100,c1=10,c2=1,kernal=2):
    super().__init__()
    self.embed=nn.EmbeddingBag(len(vocab)+1,embed)
    self.conv=nn.Sequential(
        nn.Conv1d(embed,c1,kernal),
        nn.AvgPool1d(kernal),
        nn.Conv1d(c1,c2,kernal), 
        nn.AdaptiveMaxPool1d(100)
    )
    self.fc=nn.Linear(100,mid)
    self.fc2=nn.Linear(mid,out)
  def forward(self,x):
    w=torch.stack([self.embed(word2tensor(i).unsqueeze(0)) for i in remove_stops(x.split(' '))]).permute(1,2,0).to(device)
    features=self.conv(w)
    f1=nn.functional.relu(self.fc(features))
    return torch.sigmoid(self.fc2(f1).reshape(-1))

hp=train_better(gru2,pro_train_text,pro_val_text,25,lr=2.7e-5,optim='sgd',loss='bce')


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.6 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


0/886 1/886 2/886 3/886 4/886 5/886 6/886 7/886 8/886 9/886 10/886 11/886 12/886 13/886 14/886 15/886 16/886 17/886 18/886 19/886 20/886 21/886 22/886 23/886 24/886 25/886 26/886 27/886 28/886 29/886 30/886 31/886 32/886 33/886 34/886 35/886 36/886 37/886 38/886 39/886 40/886 41/886 42/886 43/886 44/886 45/886 46/886 47/886 48/886 49/886 50/886 51/886 52/886 53/886 54/886 55/886 56/886 57/886 58/886 59/886 60/886 61/886 62/886 63/886 64/886 65/886 66/886 67/886 68/886 69/886 70/886 71/886 72/886 

In [ ]:
def mean(x):
  return sum(x)/len(x)
#text_lengths=[len(remove_stops(i,True)) for i in text if not isnan(i)]
all_text=[]
for i in range(len(titles)):
  if tensorlabel[i]>-1:
    if not isnan(text[i]):
      all_text.append((text[i],tensorlabel[i]))
def zipper(x,y):
  return [(x[i],y[i]) for i in range(len(x)) if y[i]>-1 and not isnan(x[i])]


In [ ]:
gl=len(text[0].split(' '))
gls=[]
for i in text:
  if not isnan(i):
    r=len(remove_stops(i.split(' '),True))
    gls.append(r)
    if r>gl:
      gl=r
def search_len(x,l):
  for i in x:
    if not isnan(i):
      if len(remove_stops(i.split(' '),True))==l:
        print(remove_stops(i.split(' '),True))
search_len(text,9

['2019', 'novel', 'coronavirus', 'contains', '"pShuttle-SN"', 'sequence', 'proving', 'laboratory', 'origin']
['NIH', 'researched', 'chloroquine', 'concluded', 'effective', 'stopping', 'SARS', 'coronavirus', 'tracks']


0.5037878787878788

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
so=sorted([len(i.split(' ')) for i in text if not isnan(i)])

sorted_text_lens1={len(i.split(' ')):i for i in text if not isnan(i)}
sorted_text_lens={i:sorted_text_lens1[i] for i in so}
pro_train_text[5]

('Earlier this month, the US Centers for Disease Control and Prevention updated its Covid-19 recommendations to specifically target older adults and people with serious underlying medical conditions. They labeled this group as "higher risk."However, another group of people could be particularly vulnerable to Covid-19 and hasn\'t received as much attention: people who smoke, vape or have substance use disorders."[T]he research community should be alert to the possibility that [Covid-19] could hit some populations with substance use disorders particularly hard," Dr. Nora Volkow, director of the National Institute on Drug Abuse wrote in a blog post published last week.Because Covid-19 attacks the lungs, those who smoke tobacco or marijuana or who vape may be especially threatened, Volkow said."When someone\'s lungs are exposed to flu or other infections the adverse effects of smoking or vaping are much more serious than among people who do not smoke or vape," Stanton Glantz, professor of 

In [ ]:
len(list(filter(lambda x: len(x[0])>100,pro_train_text))),len(pro_train_text)

(870, 887)

In [ ]:
import torch
cel=torch.nn.CrossEntropyLoss()
inpt=torch.randn(2,1)
tar=torch.LongTensor([0,0])
cel(inpt,tar)
def gratlen(x):
  g=len(x[0].split(' '))
  for i in x:
    if not isnan(i):
      if len(i.split(' '))>g:
        g=len(i.split(' '))
    else:
      print(i)
  return g
gratlen(text)

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


5206

In [ ]:
def expand(x,depth):
  return torch.stack([x for i in range(depth)])
a=torch.randn(1,4,1)
print(a)
a.sum(dim=1)

tensor([[[ 0.8308],
         [-0.6540],
         [-0.7270],
         [ 0.3694]]])


tensor([[-0.1808]])

In [ ]:
a='''Washington benefits from new SARS unsettling major competitors. Prior to the advent of the 2019-nCoV coronavirus, the so-called “first package” of the US-China trade agreement, which was signed on behalf of the Celestial Empire by Vice-Premier of the State Council of the PRC Liu He, was a tactical move that would allow China to take a break. And there is the presidential election in the United States, and it remains to be seen whether Donald Trump will be able to gain a foothold in the Oval Office for a second term ... But it looks like a malicious virus that has matured either in the stomachs of bats or in a snake’s gut has confused all the cards to a friend Xi Jinping.Against the background of alarming reports from the Chinese province of Hubei (when this material was being prepared, more than 4.5 thousand people fell ill with pneumonia, and 106 died, and these are not final numbers. - Auth.) Copper and iron ore prices collapsed.Copper cheaper ten days in a row, and yesterday the price fell by 1.5% - up to 5834.5 dollars per ton. Iron ore futures are falling in price, on Tuesday at the Singapore auction they offered 85 dollars per ton - a decrease of 6.4%. But, as Bloomberg predicts, $ 80 per tonne is not such a distant prospect. Prices for black gold also plummeted - April futures for the North Sea Brent oil mix were already trading below $ 58 per barrel.At the same time, the Fitch rating agency cautiously reports that the spread of coronavirus will first of all have a negative impact on the economies of Thailand, Vietnam and Singapore, since in these countries the most vulnerable regions to the epidemics are tourism and the service sector. The statement is indisputable - everyone will get it, and yet China, presumably, is more than others, since it is the Chinese economy that is the main consumer of both metals and energy resources, and tourism with services is not the last source of replenishment of the treasury. And since the prices of these goods went down, the Chinese economy also caught the “virus”. And it’s easier to negotiate with a “sick” client, which is well known even without Trump.It is noteworthy that 2019-nCoV got to the American technological giant Apple, whose production facilities are located just 500 kilometers from the “homeland” of the coronavirus - the Wuhan metropolis. And now, according to the Nikkei Asian Review, the production of popular iPhone smartphones is in jeopardy. Starbucks also found itself in a difficult situation, which, due to quarantine, had to close a large half of its four thousand coffee houses operating in China (by the way, this is the second largest market after the United States). But at the same time, the Americans for some reason do not look like such victims of the epidemic, especially Apple, whose leadership at the very beginning of the trade and economic confrontation with China advised to evacuate production facilities in the United States. It seems as if President Trump saw everything in advance, like Wolf Messing or old Wang. However, both Apple and Starbucks were really out of luck. You can’t say the same about the American political establishment, which is making titanic efforts to bring to its knees the "presumptuous China." And why should these efforts necessarily be economic in nature? The main question is - who benefits from another SARS unsettling a competitor? If we apply the well-known trick highly likely, unceremoniously used by the British ex-prime minister Theresa May to hang the so-called Skripals poisoning in Russia, then the answer is obvious: the coronavirus epidemic 2019-nCoV, which hit China, is highly likely on the hand of the United States. And the worse for Beijing, the better for Washington. Moreover, the outlook for the epidemic is not yet encouraging. For example, some experts argue: if the spread of 2019-nCoV cannot be stopped, up to 250 million Chinese can become victims of the virus, which is almost two in number in Russia.By the way, about the prospects. The Cepi Global Vaccine Coalition has reportedly invested a total of $ 12.5 million in three projects, in which researchers are ready to expeditiously develop the 2019-nCoV vaccine.Scientists from the Australian University of Queensland, as well as two American biotechnology companies Inovio and Moderna, participate in the work. The American National Institute of Allergy and Infectious Diseases (NIAID) is also involved. And it seems that microbiologists from Hong Kong have already developed a vaccine against the new coronavirus, as Professor of Hong Kong University Ewan Kwok-Jung hastened to inform the newspaper South China Morning Post. But her research, he claims, could take more than a year. German Shipulin, the deputy director of the Center for Strategic Planning of the Russian Ministry of Health, does not promise an imminent victory over 2019-nCoV. So the hands of the coronavirus 2019-nCoV can be considered as yet untied, and there are suspicions that the United States may be involved in this epidemic. In the Russian media, this assumption has already been made.
US operations as part of a biological warfare against the entire planet.As a result of the dengue epidemic in Cuba from 1978-1981, up to 500 thousand people were affected. Fidel Castro said this was the result of an American biological attack. Washington did not confirm or refute the allegations of the Cuban leader.Pentagon bio laboratories exist in 25 countries around the world. They are funded by the Military Threat Reduction Agency under a $ 2.1 billion military program. The joint biological interaction program includes laboratories located in the countries of the former Soviet Union, such as Georgia, Ukraine, Azerbaijan, Uzbekistan and Kazakhstan, as well as in the Middle East, Southeast Asia and Africa. Washington did not confirm or refute this data.By the location of these biological laboratories, we can confidently indicate the four countries and territories against which the American biological threats are now directed, these are Russia, Iran, China and the countries of Central and West Africa. Washington did not confirm or refute this data.The American company CH2M Hill, under contracts for the Pentagon bio-laboratories in Georgia, Uganda, Tanzania, Iraq, Afghanistan, Southeast Asia, was funded in the amount of $ 341.5 million. Of this amount, almost half ($ 161.1 million) was allocated for research in Lugar Center in Tbilisi. Washington did not confirm or refute this data.Its subcontractor, the private company Battelle, has worked in Pentagon bio labs in Afghanistan, Armenia, Georgia, Uganda, Tanzania, Iraq, Afghanistan and Vietnam. Battelle conducts research, development, testing and evaluation of the use of both highly toxic chemicals and highly pathogenic biological agents for a wide range of US government agencies. The company entered into federal contracts totaling $ 2 billion and ranks 23rd in the list of 100 best US state contractors. Washington did not confirm or refute this data.The Pentagon has a very long history of using insects as carriers of disease. According to a partially declassified US Army report of 1981, American scientists conducted a series of experiments on insects. These operations were part of the US Entomological War as part of a biological weapons program. Washington did not confirm or refute this data.The report reported two scenarios: 16 simultaneous attacks on the city by A. Aegypti mosquitoes infected with yellow fever, as well as Tularemia aerosol attack, and assessed their effectiveness in cash and loss of life. The results were very cynical. Pentagon experts were able to "kill" 625 thousand people at a cost of $ 0.29 per unit. Washington did not confirm or refute this data.Operation Big Itch: Field trials were carried out to determine the coverage and survivability of tropical rat fleas Xenopsylla cheopis for use as a disease carrier in biological warfare. In Ashington neither confirmed nor refuted this information.Operation Big Buzz: 1 million A. Aegypti mosquitoes were raised. One third of them were placed in ammunition, dropped from aircraft and scattered on the ground. Mosquitoes survived in the air and actively sought human blood. Washington did not confirm or refute this data.Operations on military experiments with tropical mosquitoes and ticks in Georgia. These types of mosquitoes and fleas (which were studied in the past as part of the US Entomological War Program) were imported into Georgia and tested at the Lugar Center. Washington did not confirm or refute this data.Anthrax is one of the biological agents in service with the US Army, not only in the past. Despite Pentagon claims that this program is only defensive, there are facts to the contrary. In 2016, at the Lugar Center, American scientists conducted the study “Genome Sequence of the Soviet / Russian Bacillus anthracis Vaccine Strain 55-VNIIVViM” (the sequence of the Soviet / Russian anthracite cancer vaccine in Russia), which was funded by the US Agency for Biological Weapons Sharing Program Threat Reduction (DTRA) in Tbilisi, and implemented by Metabiota. Washington did not confirm or refute this data.Congo-Crimean hemorrhagic fever (CCHF) is caused by the tick-borne virus Nairovirus (Nairovirus). The disease was first described in Crimea in 1944 and was called Crimean hemorrhagic fever. Later, it caused an epidemic in the Congo in 1969. In 2014, 34 people were infected with the CCHF, including a 4-year-old child, three of whom died. Pentagon biologists are currently studying the virus in Georgia as part of the DTRA project, Epidemiology of febrile illnesses caused by Dengue viruses and other Arboviruses in Georgia. The project included trials on patients with fever symptoms and collection of ticks as possible distributors of CCHV for laboratory analysis. Washington did not confirm or refute this data.Similar CCHV outbreaks occurred in Afghanistan, where there are 3 Pentagon bio labs. As of December 2017, 237 cases of CCHV were reported in this country, 41 of which were fatal. Washington did not confirm or refute this data.Bats are also being investigated as a carrier of various diseases at the Lugar Center, which scientists say are carriers of the Ebola virus, Middle East respiratory syndrome (MERS) and other deadly diseases. As of June 2017, 1980 cases were recorded with 699 deaths in 19 countries worldwide caused by MERS-CoV. This virus is designed and manufactured in the USA. Washington did not confirm or refute this data.Another weapon of bioterrorism is, according to a 1981 US Army report, Tularemia developed in the USA, or Rabbit Fever. New tularemia carriers, such as ticks and rodents, are currently being developed. DTRA has launched a number of projects on Tularemia and in Georgia at the Lugar Center. Highly pathogenic agents (EDP) can be used for military purposes. Washington did not confirm or refute this data.
Ukraine itself does not have control over military biological laboratories on its territory. Under the 2005 Agreement between the US Department of Defense and the Department of Health of Ukraine, the Ukrainian government is prohibited from publicly disclosing confidential information about the US program. Ukraine is also required to transfer all dangerous pathogens to the US Department of Defense for biological research. Under this Agreement, the Pentagon was granted access to many state secrets of Ukraine. Washington did not confirm or refute this data.The Pentagon has invested at least $ 65 million in gene editing research. The U.S. Department of Defense Advanced Research Projects Agency (DARPA) has provided seven research teams to develop tools for altering the genome of insects, rodents, and bacteria through the DARPA Safe Gene program using new CRISPR-Cas9 technology. Washington did not confirm or refute this data.The worst biological weapon that may have already been used and possibly used in Russia is, again, perhaps used against the “enemy” of the USA - China. Until recently, ethnic biological weapons (biogenetic weapons) were theoretical weapons, the purpose of which is, first of all, to harm people of certain ethnic groups or genotypes (Russians, Chinese, etc.). Although officially the research and development of ethnic bio-weapons has never been publicly confirmed, documents show that the United States collects biological materials from certain ethnic groups - Russians and Chinese. American National Socialism in its purest form.US Air Force specifically collects samples of Russian DNA and synovial tissue, which causes concern in Moscow about a hidden American program for the use of biological weapons.Senator Franz Klintsevich commented on the words of President Vladimir Putin about the purposeful collection of Russian biomaterial: “In the West, everything is done very scrupulously and verified to the smallest detail: if we use biological weapons, then surely ... The relevant services in the West should know that we are aware of their interest. Let those who are engaged in this work on the territory of the Russian Federation not be offended. ”And it cannot be said that such suspicions have no reason. As you know, the United States ratified the Geneva Protocol and the Biological Weapons Convention back in 1975. But the biological games overseas have not stopped, and not only in the national territory. Already after the collapse of the USSR, American biological laboratories appeared, which is precisely established in Georgia, Ukraine, Kazakhstan, Azerbaijan and Uzbekistan. Where else - except that the State Department knows that it is, they say, absolutely peaceful organizations involved in the development of medicines. But if they are so peaceful, then why, one wonders, did the Americans build them not at home, but at the other end of the world?And the participants of the “visiting” American biological project are very specialized. For example, the US Army Institute of Infectious Disease Medical Research (USAMRIID, Fort Detrick), the Military Threat Reduction Agency (DTRA), which is a division of the Pentagon, the Central Asian and Caucasian Biosafety Association, which monitors the biological potential of the CIS countries, the Biological Threat Reduction Program (Nunn-Lugar program) and so on.Comments, as they say, are unnecessary. Not so long ago, the Ministry of Defense of the Russian Federation analyzed documents on the activities of the so-called "health center" that the Americans built in Georgia, and came to the conclusion: in fact, this is a "death factory", which killed 73 people who were used by overseas biologists as experimental rabbits. But the fact of the matter is that American bio-laboratories are scattered not only around Russia, but also around the world, where America has its own interests.And another question, where did the 2019-nCoV coronavirus originate - in a bat or in some American "health center"? Where is it more highly likely?'''

In [ ]:
e=nn.EmbeddingBag(len(vocab)+1,3)
o=[]
for i in a.split(' '):
  o.append(e(word2tensor(i).unsqueeze(0)))
len(o)
del o,a